In [1]:
#importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
#putting movies data on 'movies' dataframe
data = pd.read_csv('Superpowers.csv')

ModuleNotFoundError: No module named 'numpy'

In [ ]:
data['Professional Bio'][0]

In [ ]:
# Pre-processing step: remove words like we'll, you'll, they'll etc.
data['Professional Bio'] = data['Professional Bio'].replace({"'ll": " "}, regex=True)
# Another Pre-preprocessing step: Removal of '-'
data['Professional Bio'] = data['Professional Bio'].replace({"-": " "}, regex=True)
# Remove all characters except numbers & alphabets
data['Professional Bio'] = data['Professional Bio'].replace({"[^A-Za-z0-9 ]+": ""}, regex=True)
# Another pre-processing
data['Professional Bio'] = data['Professional Bio'].fillna('')

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
bio_matrix = tfidf.fit_transform(data['Professional Bio'])
#Output the shape of tfidf_matrix
bio_matrix.shape
#Output
(45466, 75827)

In [ ]:
# true_k, derived from elbow method and confirmed from pluralsight's website
true_k = 8
# Running model with 15 different centroid initializations & maximum iterations are 500
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=15)
model.fit(bio_matrix)

In [ ]:
# elbow test
import matplotlib.pyplot as plt
# Continuing after vectorization step
# data-structure to store Sum-Of-Square-Errors
sse = {}
# Looping over multiple values of k from 1 to 40
for k in range(1, 40):
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=100).fit(bio_matrix)
    data["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_
# Plotting the curve with 'k'-value vs SSE
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")

In [ ]:
# true_k, derived from elbow method and confirmed from pluralsight's website
true_k = 30
# Running model with 15 different centroid initializations & maximum iterations are 500
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=15)
model.fit(bio_matrix)

In [ ]:
def cluster_predict(str_input):
    Y = tfidf.transform(list(str_input))
    prediction = model.predict(Y)
    return prediction

In [ ]:
# Create new column for storing predicted categories from our trained model.
data['ClusterPrediction'] = ""
# Cluster category for each live course
data['ClusterPrediction']=data.apply(lambda x: cluster_predict(data['Professional Bio']), axis=0)

In [ ]:
def recommend_util(str_input):
    
    # match on the basis course-id and form whole 'Description' entry out of it.
    temp_df = data.loc[data['Cornell Tech Email (Required)'] == str_input]
    str_input = temp_df['Professional Bio']
    
    # Predict category of input string category
    prediction_inp = cluster_predict(str_input)
    prediction_inp = int(prediction_inp)
    # Based on the above prediction 10 random students are recommended from the whole data-frame
    # Recommendation Logic is kept super-simple for current implementation.
    temp_df = data.loc[data['ClusterPrediction'] == prediction_inp]
    temp_df = temp_df.sample(10)

    temp_df['full name'] = temp_df[['Preferred First Name (Required)', 'Last Name / Family Name (Required)']].agg(' '.join, axis=1)
    
    return list(temp_df['full name'])

In [ ]:
queries = ['xd42@cornell.edu']
for query in queries:
    res = recommend_util(query)
    print(res)